<a href="https://colab.research.google.com/github/IdealAdarsh9/Multi_agent_trading/blob/main/Algorithmic_Trading_Council.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf

In [ ]:
import os
import time
import json
import random
import datetime
import google.generativeai as genai
from dataclasses import dataclass
from typing import List, Dict, Any
from enum import Enum
import yfinance as yf

# ==========================================
# 1. CONFIGURATION
# ==========================================
# 🔑 REPLACE WITH YOUR ACTUAL API KEY
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "Your_api_key")

if not GEMINI_API_KEY:
    print("⚠️  GEMINI_API_KEY not found. Please set it or input below.")
    # GEMINI_API_KEY = input("Enter your Gemini API Key: ")

genai.configure(api_key=GEMINI_API_KEY)

# --- AUTO-DETECT MODEL ---
def get_optimal_model_name():
    print("🔍 Auto-detecting best available Gemini model...")
    try:
        available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
        priorities = [
            "models/gemini-2.5-flash", # Priority 1
            "models/gemini-1.5-flash",
            "models/gemini-1.5-flash-001",
            "models/gemini-1.5-pro",
            "models/gemini-pro"
        ]
        for p in priorities:
            if p in available_models:
                print(f"✅ Selected Model: {p}")
                return p
        fallback = available_models[0] if available_models else "models/gemini-1.5-flash"
        return fallback
    except Exception as e:
        print(f"⚠️ Model list failed ({e}). Defaulting to 'models/gemini-2.5-flash'")
        return "models/gemini-2.5-flash"

MODEL_NAME = get_optimal_model_name()

GENERATION_CONFIG = {
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 2048,
    "response_mime_type": "application/json",
}

# ==========================================
# 2. DATA STRUCTURES
# ==========================================
@dataclass
class AssetData:
    ticker: str
    price: float
    change_24h: float
    market_cap: str
    volume_24h: str
    rsi_14: int
    macd_signal: str
    trend_7d: str
    news_headlines: List[str]
    last_updated: str

# ==========================================
# 3. REAL-TIME DATA PROVIDER
# ==========================================
class MarketDataProvider:
    def __init__(self, ticker):
        crypto_list = ["BTC", "ETH", "SOL", "XRP", "ADA", "DOGE", "DOT", "MATIC"]
        if ticker in crypto_list:
            self.ticker = f"{ticker}-USD"
        else:
            self.ticker = ticker

    def fetch_latest_data(self) -> AssetData:
        try:
            stock = yf.Ticker(self.ticker)
            price = stock.fast_info.last_price
            prev_close = stock.fast_info.previous_close
            change = ((price - prev_close) / prev_close) * 100

            news = []
            if stock.news:
                for n in stock.news[:3]:
                    title = n.get('title') or n.get('content', {}).get('title') or "News item available"
                    news.append(title)
            if not news:
                news = ["No recent news found."]

        except Exception as e:
            print(f"⚠️ Market Data Error: {e}")
            price = 100.00
            change = 0.0
            news = ["Market data unavailable."]

        timestamp = datetime.datetime.now().strftime("%H:%M:%S")

        return AssetData(
            ticker=self.ticker,
            price=round(price, 2),
            change_24h=round(change, 2),
            market_cap="Real-Time",
            volume_24h="Real-Time",
            rsi_14=50,
            macd_signal="Neutral",
            trend_7d="Real-Time",
            news_headlines=news,
            last_updated=timestamp
        )

# ==========================================
# 4. AI ANALYST AGENTS (With Rate Limiting)
# ==========================================
class BaseAgent:
    def __init__(self, role):
        self.role = role
        self.model = genai.GenerativeModel(
            model_name=MODEL_NAME,
            generation_config=GENERATION_CONFIG
        )

    def process(self, prompt):
        # RATE LIMIT HANDLING: Wait before request
        time.sleep(4)

        try:
            return json.loads(self.model.generate_content(prompt).text)
        except Exception as e:
            # If 429 happens even after sleep, wait longer and retry once
            if "429" in str(e):
                print(f"⏳ Rate limit hit in {self.role}. Retrying in 10s...")
                time.sleep(10)
                try:
                    return json.loads(self.model.generate_content(prompt).text)
                except:
                    pass

            print(f"⚠️ Error in {self.role}: {e}")
            return {
                "signal": "HOLD",
                "reason": "API Error/Timeout",
                "mood": "Neutral",
                "confidence": 0.0,
                "final_action": "HOLD",
                "confidence_score": 0.0,
                "risk_assessment": "High",
                "summary": "Agent unavailable due to API error."
            }

class TechnicalAgent(BaseAgent):
    def analyze(self, data: AssetData):
        prompt = f"""
        Act as a Crypto Technical Analyst. Review {data.ticker}:
        Price: ${data.price}
        RSI: {data.rsi_14}
        MACD: {data.macd_signal}
        24h Change: {data.change_24h}%

        Is this a good entry/exit point based PURELY on charts?

        Output JSON: {{ "signal": "BUY|SELL|HOLD", "reason": "string", "confidence": float }}
        """
        return self.process(prompt)

class SentimentAgent(BaseAgent):
    def analyze(self, data: AssetData):
        prompt = f"""
        Act as a Crypto Sentiment Analyst. Review headlines for {data.ticker}:
        {json.dumps(data.news_headlines)}

        Determine the 'Market Mood' (Fear vs Greed).

        Output JSON: {{ "mood": "Fear|Greed|Neutral", "signal": "BUY|SELL|HOLD", "reason": "string" }}
        """
        return self.process(prompt)

class HeadTrader(BaseAgent):
    def decide(self, data: AssetData, tech_report, sent_report):
        tech_signal = tech_report.get('signal', 'HOLD')
        tech_reason = tech_report.get('reason', 'No data')
        sent_signal = sent_report.get('signal', 'HOLD')
        sent_mood = sent_report.get('mood', 'Neutral')

        prompt = f"""
        Act as a Head Trader. Synthesize these reports for {data.ticker}:

        Technical: {tech_signal} ({tech_reason})
        Sentiment: {sent_signal} ({sent_mood})

        Context: It is {data.last_updated}.

        Make a final decision. If signals conflict, prioritize Risk Management (HOLD).

        Output JSON:
        {{
            "final_action": "BUY|SELL|HOLD",
            "confidence_score": 0.0-1.0,
            "risk_assessment": "Low|Medium|High",
            "summary": "One sentence rationale."
        }}
        """
        return self.process(prompt)

# ==========================================
# 5. MAIN MONITORING LOOP
# ==========================================
def run_monitoring_session():
    print("\n" + "="*50)
    print("🚀 CRYPTO SENTINEL AGENT INITIALIZED")
    print("="*50)

    ticker = input("Enter asset to monitor (e.g., BTC, ETH): ").upper()
    if not ticker: ticker = "BTC"

    provider = MarketDataProvider(ticker)

    # Initialize Agents
    tech_agent = TechnicalAgent("Technical Analyst")
    sent_agent = SentimentAgent("Sentiment Analyst")
    boss = HeadTrader("Head Trader")

    print(f"\n✅ Monitoring {ticker} started. Press Ctrl+C to stop.")

    try:
        while True:
            # 1. Fetch Data
            data = provider.fetch_latest_data()
            print(f"\n\n⏱️  TIME: {data.last_updated} | 💰 PRICE: ${data.price:,.2f} ({data.change_24h}%)")
            print("-" * 50)

            # 2. Run Analysis
            print("Thinking...", end="\r")
            tech_res = tech_agent.analyze(data)
            sent_res = sent_agent.analyze(data)
            final_res = boss.decide(data, tech_res, sent_res)

            # 3. Display Output
            color = "\033[92m" if "BUY" in final_res.get('final_action', 'HOLD') else "\033[91m" if "SELL" in final_res.get('final_action', 'HOLD') else "\033[93m"
            reset = "\033[0m"

            print(f"📈 Tech Signal: {tech_res.get('signal', 'N/A')} (RSI: {data.rsi_14})")
            print(f"📰 News Mood:  {sent_res.get('mood', 'N/A')}")
            print(f"{color}📢 FINAL DECISION: {final_res.get('final_action', 'ERROR')} (Conf: {final_res.get('confidence_score', 0):.2f}){reset}")
            print(f"📝 Rationale: {final_res.get('summary', 'System Error')}")

            # 4. Wait for next cycle
            wait_time = 300 # 5 minutes
            print(f"\n💤 Sleeping for {wait_time} seconds (Refresh at +5 mins)...")
            time.sleep(wait_time)

    except KeyboardInterrupt:
        print("\n\n🛑 Monitoring Stopped by User.")

if __name__ == "__main__":
    run_monitoring_session()

🔍 Auto-detecting best available Gemini model...
✅ Selected Model: models/gemini-2.5-flash

🚀 CRYPTO SENTINEL AGENT INITIALIZED
Enter asset to monitor (e.g., BTC, ETH): sol

✅ Monitoring SOL started. Press Ctrl+C to stop.


⏱️  TIME: 17:41:18 | 💰 PRICE: $124.37 (-8.67%)
--------------------------------------------------
📈 Tech Signal: HOLD (RSI: 50)
📰 News Mood:  Fear
📢 FINAL DECISION: HOLD (Conf: 0.70)
📝 Rationale: Despite significant recent price drop and fear-driven sentiment indicating SELL, core technical indicators remain neutral, leading to a risk-managed HOLD decision.

💤 Sleeping for 300 seconds (Refresh at +5 mins)...


🛑 Monitoring Stopped by User.
